In [16]:
import pandas as pd
data = pd.read_csv('./dscan_result/data/sim_hash_result.csv')
print(data.head())

dict ={}
        
for i in range(len(data['label'])):
    if data['label'][i] not in dict:
        dict[data['label'][i]] = len(dict)
print(dict)

data.label.value_counts()

                                             feature       label
0  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...  no_version
1  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...  no_version
2  HTTP/1.1 200 OK Data: DATE Server: Masked Last...  no_version
3  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...  no_version
4  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...  no_version
{'no_version': 0, '2.4.29': 1, '2.2.22': 2, '2.4.53': 3, '2.4.25': 4, '2.4.6': 5, '2.4.52': 6, '2.4.38': 7, '2.4.7': 8, '2.2.15': 9, '2.2.9': 10, '2.0.40': 11, '2.4.41': 12, '2.4.43': 13, '2.4.54': 14, '2.4.10': 15, '2.4.35': 16, '2.2.16': 17, '2.4.18': 18, '2.2.14': 19, '2.4.51': 20, '2.2.27': 21, '1.3.39': 22, '2.4.46': 23, '2.2.11': 24, '2.4.23': 25, '2.0.63': 26, '2.4.17': 27, '2.4.34': 28, '2.2.8': 29, '2.2.31': 30, '2.2.21': 31, '2.0.54': 32, '2.2.34': 33, '2.2.3': 34, '2.0.53': 35, '2.4.33': 36, '2.4.37': 37, '2.2.4': 38, '2.2.25': 39, '2.4.39': 40, '2.4.48': 41, '2.2.17': 42, '2.0.47': 43, '2.4.

no_version      5409
2.4.6           1221
2.2.15           819
2.4.54           762
2.4.41           644
                ... 
2.x.x              1
6.6.6              1
1.0/sinksoft       1
1.3.24             1
1.3.0              1
Name: label, Length: 126, dtype: int64

In [17]:

for i in range(len(data['label'])):
    data['label'][i] = dict[data['label'][i]]

for i in range(len(data['feature'])):
    data['feature'][i] = str(data['feature'][i]).lstrip().rstrip()
print(data.head(10))

                                             feature label
0  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     0
1  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     0
2  HTTP/1.1 200 OK Data: DATE Server: Masked Last...     0
3  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     0
4  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     0
5  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     1
6  HTTP/1.1 403 Forbidden Data: DATE Server: Mask...     0
7  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     0
8  HTTP/1.1 403 Forbidden Data: DATE Server: Mask...     2
9  HTTP/1.1 200 OK Data: DATE Server: Masked Last...     0


In [18]:
# with open('./dscan_result/data/to_train.csv', 'w', encoding='utf-8') as f:
#     for i in range(len(data['label'])):
#         line = data['feature'][i] + ' , ' + str(data['label'][i])
#         f.write(line)
#         f.write('\n')
#     f.close()

In [19]:
def simpleSampling(group, frac):
    return group.sample(frac=frac)

train_df = data.groupby(data.label).apply(simpleSampling, 0.5).sample(frac=1, ignore_index = True)
print(train_df)

                                                feature label
0     HTTP/1.1 400 Bad request Content-length: 90 Ca...     9
1     HTTP/1.1 400 Bad Request Data: DATE Server: Ma...    12
2     HTTP/1.0 400 Bad request Cache-Control: no-cac...     5
3     HTTP/1.1 200 OK Data: DATE Server: Masked Last...     9
4     HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     0
...                                                 ...   ...
7800  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     0
7801  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     3
7802  HTTP/1.0 302 Found Location: http://147.47.71....     9
7803  HTTP/1.1 400 BAD_REQUEST Content-Length: CONTE...     0
7804  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...    15

[7805 rows x 2 columns]


In [20]:
data = data.append(train_df.sample(frac=1, ignore_index=True)).drop_duplicates(keep = False)
print(data)

/var/folders/lw/fpq33cgj2cd5nkrb5n9wr98c0000gn/T/ipykernel_65841/4071953820.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(train_df.sample(frac=1, ignore_index=True)).drop_duplicates(keep = False)


                                                 feature label
2      HTTP/1.1 200 OK Data: DATE Server: Masked Last...     0
4      HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     0
5      HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     1
6      HTTP/1.1 403 Forbidden Data: DATE Server: Mask...     0
9      HTTP/1.1 200 OK Data: DATE Server: Masked Last...     0
...                                                  ...   ...
15630  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     5
15632  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     2
15633  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     5
15634  HTTP/1.1 400 Bad Request Data: DATE Server: Ma...     5
15636  HTTP/1.1 400 Bad Request Server: Masked Connec...     1

[7832 rows x 2 columns]


In [21]:
val_df = data.groupby(data.label).apply(simpleSampling, 0.5).sample(frac=1, ignore_index = True)
test_df = data.append(val_df).drop_duplicates(keep=False)
train_df.to_csv('./dscan_result/data/train.csv', header=None, index=None)
val_df.to_csv('./dscan_result/data/val.csv', header=None, index=None)
test_df.to_csv('./dscan_result/data/test.csv', header=None, index=None)

/var/folders/lw/fpq33cgj2cd5nkrb5n9wr98c0000gn/T/ipykernel_65841/3888539453.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = data.append(val_df).drop_duplicates(keep=False)
